# This is a Jupyter notebook!

Jupyter notebooks are awesome, because they combine documentation, code, and results

In [1]:
import pandas as pd

In [12]:
SCRIPT = """
            requirejs.undef("handsontable_view");
            console.log("handsontablejs requirejs module loaded...")
            define(
            "handsontable_view",
            [
                "jquery",
                "underscore",
                "widgets/js/widget",
                "base/js/namespace",
                // this silently upgrades jquery
                "https://cdn.jsdelivr.net/handsontable/0.20.2/handsontable.full.min.js"
            ],
            function($, _, widget, IPython){
                "use strict";
                console.log("was required");
                // Define the HandsonTableView
                var HandsonTableView = widget.DOMWidgetView.extend({
                    render: function(){           
                        _.bindAll(this, "update", "resizeHolder", "afterChange"); 
                        this.initStyle();
                        _.delay(this.update, 300);
                        //this.resizeHolder();
                    },
                    initStyle: function(){
                        var styleUrl = "https://cdn.jsdelivr.net/handsontable/0.20.2/handsontable.full.min.css";
                        if(!$('link[href="' + styleUrl + '"]').length){
                            $("<link/>", {href: styleUrl, rel: "stylesheet"})
                                .appendTo($("head"));
                        }
                    },
                    resizeHolder : function(){
                        // TODO: kill with fire
                        this.$el.find(".wtHolder")
                            .css({width: "", height: ""});
                        if(this.model.comm_live){
                            setTimeout(this.resizeHolder, 1000);
                        }
                    },
                    ensureTable: function(){
                        if(this.ht){
                            return;
                        }
                        this.ht = new Handsontable(this.$el[0], {
                            height: 300,
                            stretchH: 'all'
                        });
                        $(this.$el).width('100%');
                        this.ht.addHook("afterRender", this.afterChange);
                        IPython.keyboard_manager.register_events(this.$el);
                    },
                    update: function() {
                        this.ensureTable();
                        var value = this.model.get("value");
                        this.ht.updateSettings({
                            colHeaders: value.columns,
                            rowHeaders: value.index,
                            contextMenu: true,
                        })
                        this.ht.loadData(value.data);
                        console.log("was updated");
                    },
                    afterChange: function(event) {
                        var data = this.ht.getData(),
                            oldValue = this.model.get("value"),
                            newValue = {
                                columns: this.ht.getColHeader(),
                                index: this.ht.getRowHeader(),
                                data: data
                            };
                        if(true || !_.isEqual(oldValue, newValue)){
                            console.log("changed from client", newValue);
                            this.model.set({value: newValue});
                            this.touch();
                        }
                    },
                });
                return {
                    HandsonTableView: HandsonTableView
                };
            });"""

Javascript(SCRIPT)

NameError: name 'Javascript' is not defined

In [2]:
import handsontablesjs

In [3]:
gl_file = r'..\Files\general_ledger.csv'

In [4]:
gl = pd.read_csv(gl_file)

In [5]:
gl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 5 columns):
GL_Number         66 non-null int64
GL_Description    66 non-null object
Transaction       66 non-null object
Amount            66 non-null int64
Date              66 non-null object
dtypes: int64(2), object(3)
memory usage: 2.7+ KB


In [10]:
ht = handsontablesjs.HandsonTable(value=gl)

In [11]:
ht

In [121]:
pd.set_option('display.float_format', '{:,.0f}'.format)

In [110]:
gl[:6]

,GL_Number,GL_Description,Transaction,Amount,Date
0,100,Cash,Initial capital contrib,20000,1/1/2017
1,310,Capital,Initial capital contrib,-20000,1/1/2017
2,100,Cash,Loan from bank,180000,1/15/2017
3,250,Note Payable,Loan from bank,-180000,1/15/2017
4,550,Payroll,Payroll,15000,1/31/2017
5,100,Cash,Payroll,-15000,1/31/2017


In [119]:
gl.pivot_table(values='Amount',
               index=['GL_Number', 'GL_Description'],
               columns='Date',
               aggfunc=sum,
               margins=True,
               margins_name='Total',
               fill_value=0).style.set_table_styles([{"selector": "td", "props": [("text-align", "right")]}])